<a href="https://colab.research.google.com/github/jwalx/Tensorflow/blob/main/07_FoodVision_Big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone project 1: Food Vision Big

# Check GPU

* Google colab offers free GPUs, however, not all of them are compatiable with mixed precision training

Google colab offers:
* K80 (not compatible)
* P100 (not compatible)
* Tesla T4 (compatible)

Knowing this, in order to use mixed precision training we need access to a Tesla T4(from with Google colab) or if we're uisng our own hardware, our GPU needs a score of 7.0+

In [ ]:
!nvidia-smi -L

## Get helper functions

In past module, we've created a bunch of helper functions to do small tasks required for our notebooks.

Rather than rewriting all stuff, we can import a script and load them in from there.


In [ ]:
#download helper functions script
!wget = "https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py"

In [ ]:
# import series of helher functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from helper_functions import create_tensorboard_callback,plot_loss_curves,compare_historys

# Use Tensorflow Datasets to download data


In [ ]:
# Get Tensorflow Datasets
import tensorflow_datasets as tfds


In [ ]:
# list all available datasets
datasets_list=tfds.list_builders()
print("food101" in datasets_list)

In [ ]:
# Load in the data (will take some time)
(train_data,test_data),ds_info=tfds.load(name="food101",
                                         split=["train","validation"],
                                         shuffle_files=True,
                                         as_supervised=True,   #data gets returned in tuple format(data,label)
                                         with_info=True)

## Exploring the food101 data from tensorflow Datasets

To become one with our data, we want to find:
* Class Names
* The shape of our input data(image tensors)
* The datatype of our input data
* What the labels look like (eg: are they one-hot encoded)
* Do labels match up with the class names

In [ ]:
# Features of Food101 from TFDS
ds_info.features

In [ ]:
class_names=ds_info.features["label"].names
class_names[:10]

In [ ]:
# Take one sample of the train data
train_one_sample=train_data.take(1)


In [ ]:
# What does one sample of our training data looks like
train_one_sample

In [ ]:
# output info about our training sample
for image,label in train_one_sample:
  print(f"""
  Image shape:{image.shape}
  Image datatype: {image.dtype}
  Target class from Food101 (tensor form):{label})
  Class name (str form):{class_names[label.numpy()]}
  """)

In [ ]:
#what does our image tensor from TFDS's Food101 look like
image

In [ ]:
# What are the min and max values of TFDS's Food101
tf.reduce_min(image),tf.reduce_max(image)

# Plot an image from Tensorflow datasets


In [ ]:
# Plot an image tensor
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False);   # add title to image to verify the label associated with the image

## Create preprocessing function for our data

Neural networks perfom best when data is in a certain way( eg: batched,normalized,etc).

However, not all data (including data from Tensorflow Datasets) comes like this.

So in order to get it ready for a neural network, you'll often have to write preprocessing fucntions and mpa it to your data.

What we know about our data:
* In'unit*' datatype
* Comprised of all different size tenors (different sized images)
* Not scaled(the pixel values are between 0 & 255)

What we know models like:
* data in 'float32' dtype (or for mixed precision 'float16' and 'float32')
* For batch, Tensorflow likes all of the tensors within a batch to be the same size
* Scaled(values between 0 & 1) also called normalized tensors generally perform better

With these points in mind,we've got a few things we can tackle with a preprocessing function.

since we're going to be using an EfficientNetBX pretrained model from tf.keras.applications we don't need to rescale our data(these architecture have recaling built-in).

This means our functions need to:

1. Reshape our images to all the same size
2. Convert the dtype of our image tensors from `uint8` to `float32`

In [ ]:
# make a function fro preprocessing images
def preprocessing_img(image,label,img_shape=224):
  """
  Convert image datatype from 'uint8' -> 'float32' and reshape image to 
  [imag_shape,img_shape,color_channel]

  """
  image=tf.image.resize(image,[img_shape,img_shape])   #reshape to img_shape
  return tf.cast(image,tf.float32),label  # return (float_32,label)

In [ ]:
preprocessed_img  =preprocessing_img(image,label)[0]
print(f"Image before preprocessing:\n {image[:2]}...,\nShape:{image.shape},\nDatatype:{image.dtype}\n")
print(f"Image after preprocessing:\n {preprocessed_img[:2]} ..,\n:Shape: {preprocessed_img.shape},\nDatatype:{preprocessed_img.dtype}")

In [ ]:
# we can still plot the image as long we divide by 255.(for matplotlib compatibility)
plt.imshow(preprocessed_img/255.)
plt.title(class_names[label])
plt.axis(False)

In [ ]:
# Map preporcessing function to training data(and parallelize)
train_data=train_data.map(map_func=preprocessing_img,num_parallel_calls=tf.data.AUTOTUNE)
#shuffle train data and turn them into batches and prefetch it(helps loading it faster)
train_data=train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing fuction to test data(and parallelize it)
test_data=test_data.map(map_func=preprocessing_img,num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches and prefetch it(helps loading it faster)
test_data=test_data.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data,test_data

In [ ]:
from helper_functions import create_tensorboard_callback

# create modelcheckpoint callback to save model;s progress
checkpoint_path="model_checkpoint/cp.cpkt"  # saving weights require ".cpkt" extension
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                    monitor="val_accuracy",
                                                    save_best_only=True,
                                                    save_weights_only=True,
                                                    verbose=0)


## Setting up mixed precision

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")

In [ ]:
mixed_precision.global_policy()

In [ ]:
from tensorflow.keras import layers

input_shape=(224,224,3)
base_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False   #freeze the model

inputs=tf.keras.layers.Input(shape=input_shape,name="input_shape")

# Since we are using EfficientNet we dont require rescaling it since it already contains rescaling layer in it
# x=layers.Rescaling(1./255)(x)   

x=base_model(inputs,training=False)
x=layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x=layers.Dense(len(class_names))(x)

outputs=tf.keras.layers.Activation("softmax",dtype="float32",name="softmax_float32")(x)
model=tf.keras.Model(inputs,outputs)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])



In [ ]:
model.summary()

# Checking the dtype policies(are we using mixed precision)


In [ ]:
for layer in model.layers:
  print(layer.name,layer.trainable,layer.dtype,layer.dtype_policy)

# Fit the feature extraction model with callbacks

1. Build a feature extraction model,(general order of doing things is:)
2. Fine-tuning some of its frozen layers

In [ ]:
# fit the feature extraction model with callbacks
history_10_food_classes_feature_extract = model.fit(train_data,
                                                    epochs=3, 
                                                    steps_per_epoch=(len(train_data)),
                                                    validation_data=test_data,
                                                    validation_steps=int(0.15 * len(test_data)),
                                                    callbacks=[create_tensorboard_callback(dir_name="training_logs",
                                                                                           experiment_name="efficientnetb0_101_classes_feature_extraction"),
                                                               model_checkpoint]
                                                    )